In [3]:
! export JAX_PLATFORMS=cpu

/gpfs/cfel/group/cmi/common/psi4/psi4conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


# Rovibrational matrix elements of Cartesian tensor operators

Compute rovibrational matrix elements of spin-rotation tensors on $\text{H}_1$ and $\text{H}_2$ atoms and electric dipole moment of $\text{H}_2\text{S}$.

Here, calculations will be performed only for rotational cluster states.

In [4]:
from typing import Dict, List

import h5py
import numpy as np
from jax import config
from jax import numpy as jnp
from rovib.cartens import CART_IND
from rovib.symtop import threej_wang

config.update("jax_enable_x64", True)


The rovibrational matrix elements of a Cartesian tensor operator $T$ of rank $\Omega$ are calculated using the following formula

$$
T_{\omega}^{(J',l',J,l)}=\sum_{v',k'}\sum_{v,k}c_{v',k'}^{(J',\Gamma',l')*}c_{v,k}^{(J,\Gamma,l)}(-1)^{k'}\sum_{\sigma=-\omega}^{\omega}\sum_{\alpha}
\left(\begin{array}{ccc}
J & \omega & J' \\
k & \sigma & -k'\\
\end{array}\right)
U_{\omega\sigma,\alpha}^{(\Omega)}\langle v'|\bar{T}_{\alpha}|v\rangle.
$$

Here, $\omega=0..\Omega$, $\alpha=x,y,z$ and $xx,xy,xz,...zz$ for $\Omega=1$ and 2, respectively,
$\bar{T}_{\alpha}$ is the electronic tensor (e.g., dipole moment, see `h2s_dipole.py`, or spin-rotation tensor, see `h2s_spinrot.py`),
$v$ and $k$ denote the vibrational and rotational quanta,
$J$ and $\Gamma$ denote 'good' quantum numbers of the rotational angular momentum and the state symmetry, and $l$ is the state running index within
a set of states with the same $J$ and $\Gamma$.
The rovibrational state expansion coefficients $c_{v,k}^{(J,\Gamma,l)}$
are computed in `h2s_rovib.ipynb` and stored in seperate files
for different values of $J$ quanta.
The matrix $U_{\omega\sigma,\alpha}^{(\Omega)}$ transforms Cartesian tensor of rank $\Omega$ into its spherical-tensor representation
(see `cartens.UMAT_CART_TO_SPHER`).

The rotational part of the expression
$$
R_{\omega,k',k,\alpha\beta}^{(J',J)} = \sum_{\sigma=-\omega}^{\omega}
\left(\begin{array}{ccc}
J & \omega & J' \\
k & \sigma & -k'\\
\end{array}\right)
U_{\omega\sigma,\alpha\beta}^{(\Omega)}
$$
is computed by the `symtop.threej_wang` function where it is transformed from the symmetric-top basis $|J,k\rangle$ ($k=-J..J$) into the so-called Wang's symmetry-adapted basis $|J,k,\tau\rangle$ ($k=0..J$, $\tau=0,1$ is partity as $(-1)^\tau$).

The vibrational matrix elements
$$
V_{v',v,\alpha} = \langle v'|\bar{T}_{\alpha}|v\rangle
$$
are computed in `h2s_rovib.ipynb` and stored in file.

-------

Define functions to load the energies and quantum numbers for selected rovibrational states (`rovib_states`) and to compute the above matrix elements (`tensor_rovib_me`).

In [5]:
def rovib_states(j: int, state_ind_list: Dict[str, List[int]] = None, pmax: int = 20):
    """Reads rovibrational energies and quanta stored in files for different values
    of the J quantum number (the files are created in `h2s_rovib.ipynb`).

    Optionally, `state_ind_list[symmetry][<list of state indices>]`
    can be used to select specific states by their indices
    for different symmetries.
    """
    h5 = h5py.File(f"h2s_coefficients_pmax{pmax}_j{j}.h5", "r")
    energies = {}
    quanta = {}
    vib_indices = {}
    rot_indices = {}
    coefficients = {}
    for sym in h5["energies"].keys():
        enr = h5["energies"][sym][:]
        coefs = h5["coefficients"][sym][:]
        vind = h5["vib-indices"][sym][:]
        rind = h5["rot-indices"][sym][:]
        qua = np.array(
            [elem[0].decode("utf-8").split(",") for elem in h5["quanta"][sym][:]]
        )
        if state_ind_list is not None and sym in state_ind_list:
            energies[sym] = enr[state_ind_list[sym]]
            quanta[sym] = qua[state_ind_list[sym]]
            coefficients[sym] = coefs[:, state_ind_list[sym]]
        else:
            energies[sym] = enr
            quanta[sym] = qua
            coefficients[sym] = coefs
        vib_indices[sym] = vind
        rot_indices[sym] = rind
    return energies, quanta, vib_indices, rot_indices, coefficients

In [6]:
def tensor_rovib_me(
    rank: int,
    j1: int,
    j2: int,
    vib_me: np.ndarray,
    state_ind_list1: Dict[str, List[int]] = None,
    state_ind_list2: Dict[str, List[int]] = None,
    pmax: int = 20,
    linear: bool = False,
):
    """Computes rovibrational matrix elements of a Cartesian tensor operator.

    This function calculates the rovibrational matrix elements using wavefunctions stored in
    separate files for different values of the J quantum number (the files are creaed in `h2s_rovib.ipynb`).
    The computation is performed for the specified bra (`j1`) and ket (`j2`) values of the J
    quantum number.

    Optionally, `state_ind_list1[symmetry][<list of state indices>]` and `state_ind_list2[...]`
    can be used to select specific bra and ket states by their indices for different symmetries.
    """

    # determine the order of Cartesian indices in the Cartesian-to-spherical tensor
    #   transformation matrix (in cartens.CART_IND and symtop.threej_wang)
    cart_ind = [["xyz".index(x) for x in elem] for elem in CART_IND[rank]]

    # reshape vibrational matrix elements such that the order of Cartesian indices
    #   correspond to the order in symtop.threej_wang output
    if rank == 1:
        vib_me2 = jnp.moveaxis(jnp.array([vib_me[:, :, i] for (i,) in cart_ind]), 0, -1)
    elif rank == 2:
        vib_me2 = jnp.moveaxis(
            jnp.array([vib_me[:, :, i, j] for (i, j) in cart_ind]), 0, -1
        )
    else:
        raise ValueError(
            f"Index mapping for tensor of rank = {rank} is not implemented"
        )

    # compute rotational matrix elements of three-j symbol contracted with
    #   Cartesian-to-spherical tensor transformation matrix
    jktau_list1, jktau_list2, rot_me = threej_wang(rank, j1, j2, linear=linear)
    # rot_me[omega].shape = (2*j1+1, 2*j2+1, ncart)

    h5_1 = h5py.File(f"h2s_coefficients_pmax{pmax}_j{j1}.h5", "r")
    h5_2 = h5py.File(f"h2s_coefficients_pmax{pmax}_j{j2}.h5", "r")

    res = {}

    for sym1 in h5_1["energies"].keys():
        enr1 = h5_1["energies"][sym1][:]
        coefs1 = h5_1["coefficients"][sym1][:]
        vind1 = h5_1["vib-indices"][sym1][:]
        rind1 = h5_1["rot-indices"][sym1][:]
        qua1 = np.array(
            [elem[0].decode("utf-8").split(",") for elem in h5_1["quanta"][sym1][:]]
        )

        if state_ind_list1 is not None:
            if sym1 in state_ind_list1:
                ind = state_ind_list1[sym1]
                enr1 = enr1[ind]
                coefs1 = coefs1[:, ind]
            else:
                continue

        for sym2 in h5_2["energies"].keys():
            enr2 = h5_2["energies"][sym2][:]
            coefs2 = h5_2["coefficients"][sym2][:]
            vind2 = h5_2["vib-indices"][sym2][:]
            rind2 = h5_2["rot-indices"][sym2][:]
            qua2 = np.array(
                [elem[0].decode("utf-8").split(",") for elem in h5_2["quanta"][sym2][:]]
            )

            if state_ind_list2 is not None:
                if sym2 in state_ind_list2:
                    ind = state_ind_list2[sym2]
                    enr2 = enr2[ind]
                    coefs2 = coefs2[:, ind]
                else:
                    continue

            vib_me_ = vib_me2[jnp.ix_(vind1, vind2)]
            me = []
            for omega in rot_me.keys():
                me_ = jnp.einsum(
                    "ijc,ijc->ij", vib_me_, rot_me[omega][jnp.ix_(rind1, rind2)]
                )
                me.append(jnp.einsum("ik,ij,jl->kl", jnp.conj(coefs1), me_, coefs2))
            res[((sym1, sym2))] = jnp.moveaxis(jnp.array(me), 0, -1)

    return res

Read the indices of rotational cluster states for different $J$ quanta, sored in file `h2s_cluster_states.txt`. These states were identified in `h2s_clusters.ipynb` notebook.

In [7]:
print("Rotational cluster states")

cluster_state_id = {}

with open("h2s_cluster_states_id.txt", "r") as fl:
    for line in fl:
        w = line.split()
        j = [int(w[0 + i * 12]) for i in range(4)]
        id = [int(w[1 + i * 12]) for i in range(4)]
        sym = [w[3 + i * 12] for i in range(4)]
        assert all(j[0] == elem for elem in j), f"Not all J quanta are equal: {j}"
        j_ = j[0]

        if j_ >= 50:
            cluster_state_id[j_] = {}
            for sym_, id_ in zip(sym, id):
                try:
                    cluster_state_id[j_][sym_].append(id_)
                except KeyError:
                    cluster_state_id[j_][sym_] = [id_]
            cluster_state_id[j_] = dict(sorted(cluster_state_id[j_].items()))
            print(f"J = {j_}, cluster-state IDs: {cluster_state_id[j_]}")

Rotational cluster states
J = 50, cluster-state IDs: {'A1': [280], 'A2': [242], 'B1': [242], 'B2': [277]}
J = 51, cluster-state IDs: {'A1': [267], 'A2': [299], 'B1': [299], 'B2': [270]}
J = 52, cluster-state IDs: {'A1': [327], 'A2': [291], 'B1': [291], 'B2': [324]}
J = 53, cluster-state IDs: {'A1': [316], 'A2': [353], 'B1': [354], 'B2': [318]}
J = 54, cluster-state IDs: {'A1': [387], 'A2': [345], 'B1': [344], 'B2': [385]}
J = 55, cluster-state IDs: {'A1': [371], 'A2': [412], 'B1': [413], 'B2': [373]}
J = 56, cluster-state IDs: {'A1': [450], 'A2': [399], 'B1': [399], 'B2': [447]}
J = 57, cluster-state IDs: {'A1': [432], 'A2': [486], 'B1': [487], 'B2': [434]}
J = 58, cluster-state IDs: {'A1': [523], 'A2': [469], 'B1': [468], 'B2': [520]}
J = 59, cluster-state IDs: {'A1': [503], 'A2': [563], 'B1': [565], 'B2': [505]}
J = 60, cluster-state IDs: {'A1': [600], 'A2': [544], 'B1': [542], 'B2': [599]}


Using the functions above, compute matrix elements of spin-rotation tensors and dipole moment for rotational cluster states at $J=50..60$; store matrix elements in files.

In [8]:
pmax = 20

# read vibrational matrix elements of spin-rotation tensors and dipole moment

with h5py.File(f"h2s_vibme_pmax{pmax}.h5", "r") as h5:
    sr_h1_vib = h5["spin-rotation"]["h1"][:]
    sr_h2_vib = h5["spin-rotation"]["h2"][:]
    dipole_vib = h5["dipole-moment"][:]

# compute rovibrational matrix elements and store them in file

with h5py.File(f"h2s_enr_cluster_pmax{pmax}.h5", "w") as h5:
    for j in cluster_state_id.keys():
        enr, qua, vind, rind, coefs = rovib_states(j, state_ind_list=cluster_state_id[j])
        print(
            f"store energies for J = {j}, states = {[(sym, np.round(val, 6)) for sym, val in enr.items()]}"
        )
        for sym in enr:
            qua_str = [",".join(elem) for elem in qua[sym]]
            max_len = max([len(elem) for elem in qua_str])
            qua_ascii = [elem.encode("ascii", "ignore") for elem in qua_str]
            h5.create_dataset(f"energies/{j}/{sym}", data=enr[sym])
            h5.create_dataset(f"coefficients/{j}/{sym}", data=coefs[sym])
            h5.create_dataset(f"vib-indices/{j}/{sym}", data=vind[sym])
            h5.create_dataset(f"rot-indices/{j}/{sym}", data=rind[sym])
            h5.create_dataset(
                f"quanta/{j}/{sym}", (len(qua_ascii), 1), f"S{max_len}", data=qua_ascii
            )

store energies for J = 50, states = [('A1', array([24708.8189])), ('A2', array([24708.818796])), ('B1', array([24708.8189])), ('B2', array([24708.818796]))]
store energies for J = 51, states = [('A1', array([25447.364248])), ('A2', array([25447.36433])), ('B1', array([25447.364248])), ('B2', array([25447.36433]))]
store energies for J = 52, states = [('A1', array([26191.945163])), ('A2', array([26191.945055])), ('B1', array([26191.945163])), ('B2', array([26191.945055]))]
store energies for J = 53, states = [('A1', array([26942.107341])), ('A2', array([26942.110081])), ('B1', array([26942.108071])), ('B2', array([26942.110081]))]
store energies for J = 54, states = [('A1', array([27697.400168])), ('A2', array([27697.39982])), ('B1', array([27697.400168])), ('B2', array([27697.39982]))]
store energies for J = 55, states = [('A1', array([28457.345655])), ('A2', array([28457.345664])), ('B1', array([28457.345655])), ('B2', array([28457.345664]))]
store energies for J = 56, states = [('A1'

In [9]:
for j1 in cluster_state_id.keys():
    for j2 in cluster_state_id.keys():
        dj = abs(j1 - j2)

        # neglect off-diagonal coupling between different J because of the large energy gap
        if dj>0:
            continue

        if dj <= 1:
            dipole_me = tensor_rovib_me(
                1,
                j1,
                j2,
                dipole_vib,
                state_ind_list1=cluster_state_id[j1],
                state_ind_list2=cluster_state_id[j2],
            )  # Debye units
        else:
            dipole_me = {}

        if dj <= 2:
            sr1_me = tensor_rovib_me(
                2,
                j1,
                j2,
                sr_h1_vib,
                state_ind_list1=cluster_state_id[j1],
                state_ind_list2=cluster_state_id[j2],
            )
            sr2_me = tensor_rovib_me(
                2,
                j1,
                j2,
                sr_h2_vib,
                state_ind_list1=cluster_state_id[j1],
                state_ind_list2=cluster_state_id[j2],
            )
        else:
            sr1_me = {}
            sr2_me = {}

        if dipole_me or sr1_me or sr2_me:
            print(f"store matrix elements for J1 = {j1} J2 = {j2}, delta J = {dj}")
            with h5py.File(f"h2s_me_cluster_pmax{pmax}_j{j1}_j{j2}.h5", "w") as h5:
                for label, oper in zip(
                    ("dipole", "spin-rotation-H1", "spin-rotation-H2"),
                    (dipole_me, sr1_me, sr2_me),
                ):
                    for (sym1, sym2), me in oper.items():
                        print(f"{label}, sym = {(sym1, sym2)}, shape = {me.shape}, val = {me}")
                        h5.create_dataset(f"{label}/{sym1}/{sym2}", data=me)

store matrix elements for J1 = 50 J2 = 50, delta J = 0
dipole, sym = ('A1', 'A1'), shape = (1, 1, 1), val = [[[0.+0.j]]]
dipole, sym = ('A1', 'A2'), shape = (1, 1, 1), val = [[[0.+0.05745297j]]]
dipole, sym = ('A1', 'B1'), shape = (1, 1, 1), val = [[[0.-5.37638729e-16j]]]
dipole, sym = ('A1', 'B2'), shape = (1, 1, 1), val = [[[0.+0.j]]]
dipole, sym = ('A2', 'A1'), shape = (1, 1, 1), val = [[[0.-0.05745297j]]]
dipole, sym = ('A2', 'A2'), shape = (1, 1, 1), val = [[[0.+0.j]]]
dipole, sym = ('A2', 'B1'), shape = (1, 1, 1), val = [[[0.+0.j]]]
dipole, sym = ('A2', 'B2'), shape = (1, 1, 1), val = [[[0.-5.37337889e-16j]]]
dipole, sym = ('B1', 'A1'), shape = (1, 1, 1), val = [[[0.+5.37638729e-16j]]]
dipole, sym = ('B1', 'A2'), shape = (1, 1, 1), val = [[[0.+0.j]]]
dipole, sym = ('B1', 'B1'), shape = (1, 1, 1), val = [[[0.+0.j]]]
dipole, sym = ('B1', 'B2'), shape = (1, 1, 1), val = [[[0.+0.05745297j]]]
dipole, sym = ('B2', 'A1'), shape = (1, 1, 1), val = [[[0.+0.j]]]
dipole, sym = ('B2', 'A2'),